# MOP Generation

In [10]:
import numpy as np
import pandas as pd
import os

In [16]:
columns = [
    # --- Scenario Metadata ---
    "Flight_Number",
    "Lead_Pilot",
    "Scenario_within_flight",
    "Scenario_Type",
    "Autonomy_Config",
    "Num_Tactical_Comms",
    "Correct_Sort",
    "Scenario_Start_Time",

    # --- Aircraft State / Performance ---
    "Num_CMs",
    "Lead_Altitude_MSL_ft",
    "Wingman_Altitude_MSL_ft",
    "CM_Airspeed_kt",
    "Lead_Altitude_Deviation_Count",
    "Wingman_Altitude_Deviation_Count",
    "Lead_Altitude_Deviation_Integrated_ft_s",
    "Wingman_Altitude_Deviation_Integrated_ft_s",

    # --- CM Engagement Summary ---
    "Total_CMs_Intercepted",
    "Proportion_CMs_Intercepted",

    # --- Cruise Missile Engagement Details (CM1 - CM10) ---
]

# Dynamically extend for CM1–CM10
for i in range(1, 11):
    columns.extend([
        f"CM{i}_EntId",
        f"CM{i}_Interceptor Role",
        f"CM{i}_Time_to_Intercept_s",
        f"CM{i}_Time_to_Consent_s",
        f"CM{i}_Airspeed_at_Intercept_kt",
        f"CM{i}_Airspeed_Diff_at_Intercept_kt",
        f"CM{i}_Heading_at_Intercept_deg",
        f"CM{i}_CM_Heading_at_Intercept_deg",
        f"CM{i}_Heading_Diff_at_Intercept_deg",
        f"CM{i}_Altitude_at_Intercept_ft",
        f"CM{i}_Altitude_Offset_at_Intercept_ft",
        f"CM{i}_Bank_Angle_at_Intercept_deg",
        f"CM{i}_Distance_from_CM_at_Intercept_nm",
    ])

# --- SAM Summary ---
columns.extend([
    "Num_SAMs",
    "SAMs_Identified_by_Lead",
    "Proportion_SAMs_Identified",
])

# Dynamically extend for SAM1–SAM10
for i in range(1, 11):
    columns.extend([
        f"SAM{i}_EntId",
        f"SAM{i}_Time_to_ID_s",
    ])

# --- Scenario End ---
columns.extend([
    "Scenario_End_Time",
    "Scenario_Duration_s",
])


In [17]:
results_path = 'Output'
MOP_df = pd.DataFrame(columns=columns)
for file in os.listdir(results_path):
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(results_path, file))
        MOP_df = pd.concat([MOP_df, df], ignore_index=True)

In [18]:
MOP_df.head()

,Flight_Number,Lead_Pilot,Scenario_within_flight,Scenario_Type,Autonomy_Config,Num_Tactical_Comms,Correct_Sort,Scenario_Start_Time,Num_CMs,Lead_Altitude_MSL_ft,...,SAM7_EntId,SAM7_Time_to_ID_s,SAM8_EntId,SAM8_Time_to_ID_s,SAM9_EntId,SAM9_Time_to_ID_s,SAM10_EntId,SAM10_Time_to_ID_s,Scenario_End_Time,Scenario_Duration_s
0,3,Grimmer,1,C,HH,18.0,Y,2025-09-12 17:59:07.334,5,8987.758711,...,112.0,30.000,NaN,NaN,NaN,NaN,NaN,NaN,2025-09-12 18:06:27.937,440.603
1,3,Grimmer,2,D,HH,21.0,Y,2025-09-12 18:10:44.114,8,8985.541640,...,128.0,30.000,130.0,12.262,NaN,NaN,NaN,NaN,2025-09-12 18:20:56.002,611.888
2,3,Grimmer,3,C,AA,15.0,Y,2025-09-12 18:26:08.530,5,8857.920509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-09-12 18:31:57.435,348.905
3,2,Chan,1,B,HH,29.0,Y,2025-09-12 14:48:55.707,5,8714.375071,...,112.0,15.503,113.0,30.000,NaN,NaN,NaN,NaN,2025-09-12 14:58:07.141,551.434
4,2,Chan,2,C,HH,25.0,Y,2025-09-12 15:03:09.298,5,9260.064159,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-09-12 15:09:29.450,380.152


In [22]:
# for each row in MOP_df, calculate the average SAM{1-10}_Time_to_ID_s for each entry that is now NaN, and store this as Avg_SAM_ID_Time_s
MOP_df['Avg_SAM_ID_Time_s'] = MOP_df[[f'SAM{i}_Time_to_ID_s' for i in range(1, 11)]].mean(axis=1)

In [29]:
# we do not need the following columns in MOP_df: Scenario_Start_Time, Num_CMs, Lead_Altitude_MSL_ft, Wingman_Altitude_MSL_ft, CM_Airspeed_kt
cols_to_drop = ['Scenario_Start_Time', 'Num_CMs', 'Lead_Altitude_MSL_ft', 'Wingman_Altitude_MSL_ft', 'CM_Airspeed_kt',
                'Total_CMs_Intercepted', 'CM1_EntId', 'Num_SAMs', 'SAMs_Identified_by_Lead', 'Scenario_End_Time']
for i in range(1,11):
    cols_to_drop.extend([f'CM{i}_EntId', f'CM1_Interceptor Role', f'CM{i}_Airspeed_at_Intercept_kt',
                        f'CM{i}_Heading_at_Intercept_deg', f'CM{i}_CM_Heading_at_Intercept_deg', f'CM{i}_Altitude_at_Intercept_ft',
                        f'CM{i}_Bank_Angle_at_Intercept_deg', f'SAM{i}_EntId', f'SAM{i}_Time_to_ID_s'])

In [30]:
MOP_df = MOP_df.drop(columns=cols_to_drop)

In [ ]:
MOP_list = ['Flight_Number', 'Lead_Pilot', 'Scenario_within_flight', 'Scenario', 'Configuration', 'Tactical_Comms',
            'Target_Acquisition', 'Alt_Deviation_Lead', 'Alt_Deviation_Wingman', 'Alt_Dev_Integral_Lead_fts',
            'Alt_Dev_Integral_Wingman_fts', 'Prop_Intercepted', 'Scenario_Duration_s', 'Prop_SAMs_ID', 'Avg_SAM_ID_Time_s']
for i in range(1, 11):
    MOP_list.extend([
        f"CM{i}_Time_to_Intercept_s",
        f"CM{i}_Time_to_Consent_s",
        f"CM{i}_TCE_Airspeed_kt",
        f"CM{i}_TCE_Heading_deg",
        f"CM{i}_TCE_Altitude_ft",
        f"CM{i}_TCE_Distance_nm"
    ])

In [32]:
MOP_df.to_csv('CURRENT_MOPs.csv', index=False)